In [3]:
!pip install transformers

In [4]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [5]:
model = TFAutoModel.from_pretrained("bert-base-uncased")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [6]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [7]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [8]:
!pip install datasets

In [9]:
import pyarrow as pa
import pyarrow.dataset as ds
from datasets import Dataset

In [10]:
import pandas as pd
df = pd.read_csv('/kaggle/input/tweetlabel/TrainingData.tsv', sep='\t')
df.columns =['id1', 'id2', 'label', 'text']

In [11]:
df2 = pd.read_csv('/kaggle/input/tweetlabel/DevData.tsv', sep='\t')
df2.columns =['id1', 'id2', 'label', 'text']

In [12]:
df3 = pd.read_csv('/kaggle/input/tweetlabel/TestData.tsv', sep='\t', on_bad_lines='skip')
df3.columns =['id1', 'id2', 'label', 'text']

In [13]:
x_train, y_train = df['text'],df['label']
x_valid, y_valid = df2['text'],df2['label']
x_test, y_test = df3['text'],df3['label']


str_to_idx = {
    'negative':0,
    'positive':1,
    'neutral':2
}
y_train2 = list(map(lambda x: x.replace('objective-OR-neutral', 'neutral'), y_train))
y_train2 = list(map(lambda x: x.replace('objective', 'neutral'), y_train2))
y_train = [str_to_idx[c] for c in y_train2]

y_valid2 = list(map(lambda x: x.replace('objective-OR-neutral', 'neutral'), y_valid))
y_valid2 = list(map(lambda x: x.replace('objective', 'neutral'), y_valid2))
y_valid = [str_to_idx[c] for c in y_valid2]


y_test3 = list(map(lambda x: x.replace('objective-OR-neutral', 'neutral'), y_test))
y_test3 = list(map(lambda x: x.replace('objective', 'neutral'), y_test3))
y_test = [str_to_idx[c] for c in y_test3]

In [14]:
len(y_train)

8001

In [15]:
df = pd.DataFrame(list(zip(x_train, y_train)), columns = ['text', 'label'])
df2 = pd.DataFrame(list(zip(x_valid, y_valid)), columns = ['text', 'label'])
df3 = pd.DataFrame(list(zip(x_test, y_test)), columns = ['text', 'label'])

In [16]:
hg_dataset_train = Dataset(pa.Table.from_pandas(df))
hg_dataset_valid = Dataset(pa.Table.from_pandas(df2))
hg_dataset_test = Dataset(pa.Table.from_pandas(df3))

In [17]:
import datasets
tweets = datasets.DatasetDict({"train":hg_dataset_train,"valid":hg_dataset_valid,"test":hg_dataset_test})


In [18]:
tweets

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 8001
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 1380
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3136
    })
})

In [19]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [20]:
tweets_encoded = tweets.map(tokenize, batched=True, batch_size=None)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [21]:
tweets_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8001
    })
    valid: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1380
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3136
    })
})

In [22]:
# setting 'input_ids', 'attention_mask', 'token_type_ids', and 'label'
# to the tensorflow format. Now if you access this dataset you will get these
# columns in `tf.Tensor` format

tweets_encoded.set_format('tf',
                            columns=['input_ids', 'attention_mask', 'token_type_ids', 'label'])

# setting BATCH_SIZE to 64.
BATCH_SIZE = 4

def order(inp):
    '''
    This function will group all the inputs of BERT
    into a single dictionary and then output it with
    labels.
    '''
    data = list(inp.values())
    return {
        'input_ids': data[1],
        'attention_mask': data[2],
        'token_type_ids': data[3]
    }, data[0]

# converting train split of `emotions_encoded` to tensorflow format
train_dataset = tf.data.Dataset.from_tensor_slices(tweets_encoded['train'][:])
# set batch_size and shuffle
train_dataset = train_dataset.batch(BATCH_SIZE).shuffle(1000)
# map the `order` function
train_dataset = train_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

# ... doing the same for test set ...
test_dataset = tf.data.Dataset.from_tensor_slices(tweets_encoded['test'][:])
test_dataset = test_dataset.batch(BATCH_SIZE)
test_dataset = test_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)

In [23]:
inp, out = next(iter(train_dataset)) # a batch from train_dataset
print(inp, '\n\n', out)

{'input_ids': <tf.Tensor: shape=(4, 484), dtype=int64, numpy=
array([[  101,  7592,  2158, ...,     0,     0,     0],
       [  101,  2613,  2637, ...,     0,     0,     0],
       [  101,  1030,  9993, ...,     0,     0,     0],
       [  101, 17012,  1010, ...,     0,     0,     0]])>, 'attention_mask': <tf.Tensor: shape=(4, 484), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'token_type_ids': <tf.Tensor: shape=(4, 484), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])>} 

 tf.Tensor([1 2 2 2], shape=(4,), dtype=int64)


In [24]:
class BERTForClassification(tf.keras.Model):

    def __init__(self, bert_model, num_classes):
        super().__init__()
        self.bert = bert_model
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = self.bert(inputs)[1]
        return self.fc(x)

In [25]:
classifier = BERTForClassification(model, num_classes=3)

classifier.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

In [26]:
train_dataset

<_ParallelMapDataset element_spec=({'input_ids': TensorSpec(shape=(None, 484), dtype=tf.int64, name=None), 'attention_mask': TensorSpec(shape=(None, 484), dtype=tf.int64, name=None), 'token_type_ids': TensorSpec(shape=(None, 484), dtype=tf.int64, name=None)}, TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [27]:
history = classifier.fit(
    train_dataset,
    epochs=3
)

Epoch 1/3
2001/2001 [==============================] - 1039s 492ms/step - loss: 0.6789 - accuracy: 0.7008
Epoch 2/3
2001/2001 [==============================] - 981s 490ms/step - loss: 0.4144 - accuracy: 0.8326
Epoch 3/3
2001/2001 [==============================] - 980s 490ms/step - loss: 0.2517 - accuracy: 0.9058


In [28]:
classifier.evaluate(test_dataset)

784/784 [==============================] - 141s 175ms/step - loss: 0.6660 - accuracy: 0.7704


[0.6660326719284058, 0.7704081535339355]

In [ ]:
classifier.evaluate(test_dataset)

In [29]:
import pickle
pickle.dump(classifier, open("tweet-classifier-bert.pkl", "wb"))


/opt/conda/lib/python3.10/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [32]:
# ... doing the same for test set ...
valid_dataset = tf.data.Dataset.from_tensor_slices(tweets_encoded['valid'][:])
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(order, num_parallel_calls=tf.data.AUTOTUNE)
classifier.evaluate(valid_dataset)

345/345 [==============================] - 17s 37ms/step - loss: 0.6875 - accuracy: 0.7420


[0.6874618530273438, 0.7420290112495422]